In [1]:
%pip install --upgrade --quiet  langchainhub langgraph langchain-groq langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [2]:
from typing_extensions import Annotated, TypedDict, List

In [3]:
from langchain.prompts import ChatPromptTemplate

In [4]:
from langchain.chains import create_sql_query_chain

In [5]:
from langchain_groq import ChatGroq
from google.colab import userdata

llm = ChatGroq(temperature=0, model="llama-3.3-70b-versatile", api_key = userdata.get('GROQ_API_KEY'))

In [6]:
class ClassifyInput(TypedDict):
    """Classify the input type."""
    input_type: Annotated[str, "Only takes values: 'ingestion' or 'chat'"]

In [7]:
class Transaction(TypedDict):
    """Transaction details."""
    date: Annotated[str, "The date of the transaction in 'YYYY-MM-DD' format."]
    amount: Annotated[float, "The amount spent in the transaction."]
    vendor: Annotated[str, "The name of the vendor or store where the transaction occurred."]
    description: Annotated[str, "A brief description of the transaction, such as items purchased."]


## Natural Language Classifcation

In [8]:
classify_llm = llm.with_structured_output(ClassifyInput)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Classify the following query as either 'ingestion' or 'chat'."),
    ("human", "Query: {chunk}\n\nClassify the query as either 'ingestion' or 'chat'.")
])

def classify_query(chunk: str) -> ClassifyInput:
    response = classify_llm.invoke(prompt.format(chunk=chunk))

    return response['input_type']

In [ ]:
classify_query("I bought a laptop at Best Buy on 2025-02-21 for $799.99. The laptop is an HP Spectre x360.")

'ingestion'

## Schema Extraction

In [9]:
transaction_llm = llm.with_structured_output(Transaction)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the transaction details from the given text input. Focus on date, amount, vendor, and description."),
    ("human", "Text: {chunk}\n\nExtract the following details:\n- Date\n- Amount\n- Vendor\n- Description.")
])

In [10]:
def extract_transaction_details(chunk: str) -> Transaction:
    response = transaction_llm.invoke(prompt.format(chunk=chunk))
    return response

In [ ]:
extract_transaction_details("I bought a laptop at Best Buy on 2025-02-21 for $799.99. The laptop is an HP Spectre x360")

{'amount': 799.99,
 'date': '2025-02-21',
 'description': 'HP Spectre x360 laptop',
 'vendor': 'Best Buy'}

## Ingestion Pipeline

In [11]:
import os
import sqlite3
from dotenv import load_dotenv



def create_transaction_table():
    conn = sqlite3.connect('transactions.db')
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS transactions (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            date TEXT,
            amount REAL,
            vendor TEXT,
            description TEXT
        )
    ''')

    conn.commit()
    conn.close()

def insert_transaction_to_db(transaction: Transaction):
    conn = sqlite3.connect('transactions.db')
    cursor = conn.cursor()

    cursor.execute('''
        INSERT INTO transactions (date, amount, vendor, description)
        VALUES (?, ?, ?, ?)
    ''', (transaction['date'], transaction['amount'], transaction['vendor'], transaction['description']))

    conn.commit()
    conn.close()

def process_and_store_transaction(chunk: str):
    transaction = extract_transaction_details(chunk)

    insert_transaction_to_db(transaction)

create_transaction_table()


In [12]:
transaction_texts = [
    "I had dinner at The Bistro on 2025-02-13 for $25.50. The meal included pasta, salad, and a drink.",
    "I purchased a book from Amazon on 2025-02-14 for $15.00. The book was 'Python Programming for Beginners'.",
    "I filled up my car at Shell Gas Station on 2025-02-15 for $40.00. The fuel was premium unleaded.",
    "I subscribed to Netflix on 2025-02-16 for $12.99. The subscription is for one month of standard service.",
    "I bought a concert ticket for the Coldplay show on 2025-02-17 for $85.00. The ticket was for section A, row 10.",
    "I had lunch at Starbucks on 2025-02-18 for $8.75. I ordered a sandwich and coffee.",
    "I bought a jacket at Macy's on 2025-02-19 for $55.99. The jacket is from the new winter collection.",
    "I took a taxi ride on 2025-02-20 for $22.50. The ride was from downtown to the airport.",
    "I bought a laptop at Best Buy on 2025-02-21 for $799.99. The laptop is an HP Spectre x360.",

]

for tn in transaction_texts:
  process_and_store_transaction(tn)
  print("database updated")


database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated


## Checking if the records are stored or not

In [13]:
import sqlite3

def check_records_in_db():
    conn = sqlite3.connect('transactions.db')
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM transactions")
    records = cursor.fetchall()

    if records:
        for record in records:
            print(f"ID: {record[0]}, Date: {record[1]}, Amount: {record[2]}, Vendor: {record[3]}, Description: {record[4]}")
    else:
        print("No records found.")

    conn.close()

check_records_in_db()


ID: 1, Date: 2025-02-13, Amount: 25.5, Vendor: The Bistro, Description: dinner, pasta, salad, and a drink
ID: 2, Date: 2025-02-14, Amount: 15.0, Vendor: Amazon, Description: Python Programming for Beginners book
ID: 3, Date: 2025-02-15, Amount: 40.0, Vendor: Shell Gas Station, Description: premium unleaded fuel
ID: 4, Date: 2025-02-16, Amount: 12.99, Vendor: Netflix, Description: one month of standard service
ID: 5, Date: 2025-02-17, Amount: 85.0, Vendor: Unknown, Description: Concert ticket for Coldplay show, section A, row 10
ID: 6, Date: 2025-02-18, Amount: 8.75, Vendor: Starbucks, Description: a sandwich and coffee
ID: 7, Date: 2025-02-19, Amount: 55.99, Vendor: Macy's, Description: a jacket from the new winter collection
ID: 8, Date: 2025-02-20, Amount: 22.5, Vendor: taxi, Description: taxi ride from downtown to the airport
ID: 9, Date: 2025-02-21, Amount: 799.99, Vendor: Best Buy, Description: HP Spectre x360 laptop


## QnA (NL2SQL Pipeline)

In [14]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///transactions.db")
print(db.dialect)
print(db.get_usable_table_names())


sqlite
['transactions']


In [ ]:
answer = db.run("SELECT*FROM transactions")

In [ ]:
answer

'[(1, \'2025-02-13\', 25.5, \'The Bistro\', \'pasta, salad, and a drink\'), (2, \'2025-02-14\', 15.0, \'Amazon\', \'Python Programming for Beginners book\'), (3, \'2025-02-15\', 40.0, \'Shell Gas Station\', \'premium unleaded fuel\'), (4, \'2025-02-16\', 12.99, \'Netflix\', \'one month of standard service\'), (5, \'2025-02-17\', 85.0, \'Unknown\', \'Concert ticket for Coldplay show, section A, row 10\'), (6, \'2025-02-18\', 8.75, \'Starbucks\', \'a sandwich and coffee\'), (7, \'2025-02-19\', 55.99, "Macy\'s", \'a jacket from the new winter collection\'), (8, \'2025-02-20\', 22.5, \'taxi\', \'taxi ride from downtown to the airport\'), (9, \'2025-02-21\', 799.99, \'Best Buy\', \'HP Spectre x360 laptop\')]'

In [15]:
from langchain.chains import create_sql_query_chain
generate_query = create_sql_query_chain(llm, db)
query = generate_query.invoke({"question": "what is my total amount spending?`"})

In [16]:
query

'Question: what is my total amount spending?\nSQLQuery: SELECT SUM("amount") AS "total_spending" FROM transactions'

In [19]:
query_text = query.split("SQLQuery:")[1].strip()
print(query_text)

SELECT SUM("amount") AS "total_spending" FROM transactions


In [20]:
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool
execute_query = QuerySQLDatabaseTool(db=db)
execute_query.invoke(query_text)


'[(1065.72,)]'

In [25]:
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

answer_prompt = PromptTemplate.from_template(
    "Given the following user question, corresponding SQL query, and SQL result, answer the user question.\n\n"
    "Question: {question}\n"
    "SQL Query: {query}\n"
    "SQL Result: {result}\n"
    "Answer:"
)
rephrase_answer = answer_prompt | llm | StrOutputParser()

def answer_question(question: str, llm, db):
    generate_query = create_sql_query_chain(llm, db)
    generated_response = generate_query.invoke({"question": question})
    query_text = generated_response.split("SQLQuery:")[1].strip()
    execute_query = QuerySQLDatabaseTool(db=db)
    sql_result = execute_query.invoke(query_text)
    final_answer = rephrase_answer.invoke({
        "question": question,
        "query": query_text,
        "result": sql_result
    })
    return final_answer


In [28]:
answer_question("How much I spent on starbucks and on what items?", llm, db)

'You spent $8.75 at Starbucks on a sandwich and coffee.'